In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
from torchvision import datasets
import time

In [ ]:
# Preparing for Data
print('==> Preparing data..')
""""
# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
"""

==> Preparing data..


'"\n# Training Data augmentation\ntransform_train = transforms.Compose([\n    transforms.RandomCrop(32, padding=4),\n    transforms.RandomHorizontalFlip(),\n    transforms.ToTensor(),\n    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),\n])\n# Testing Data preparation\ntransform_test = transforms.Compose([\n    transforms.ToTensor(),\n    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),\n])\n\n#classes = (\'plane\', \'car\', \'bird\', \'cat\', \'deer\', \'dog\', \'frog\', \'horse\', \'ship\', \'truck\')\n'

In [ ]:
class LeNet(nn.Module):
     
    def __init__(self):   
        super(LeNet, self).__init__()
        # Convolution (In LeNet-5, 32x32 images are given as input. Hence padding of 2 is done below)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2, bias=True)
        # Max-pooling
        self.max_pool_1 = nn.MaxPool2d(kernel_size=2)
        # Convolution
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0, bias=True)
        # Max-pooling
        self.max_pool_2 = nn.MaxPool2d(kernel_size=2)
        # Fully connected layer
        self.fc1 = nn.Linear(16*5*5, 120)   # convert matrix with 16*5*5 (= 400) features to a matrix of 120 features (columns)
        self.fc2 = nn.Linear(120, 84)       # convert matrix with 120 features to a matrix of 84 features (columns)
        self.fc3 = nn.Linear(84, 10)        # convert matrix with 84 features to a matrix of 10 features (columns)
        
    def forward(self, x):
        # convolve, then perform ReLU non-linearity
        x = F.relu(self.conv1(x))  
        # max-pooling with 2x2 grid
        x = self.max_pool_1(x)
        # convolve, then perform ReLU non-linearity
        x = F.relu(self.conv2(x))
        # max-pooling with 2x2 grid
        x = self.max_pool_2(x)
        # first flatten 'max_pool_2_out' to contain 16*5*5 columns
        # read through https://stackoverflow.com/a/42482819/7551231
        x = x.view(-1, 16*5*5)
        # FC-1, then perform ReLU non-linearity
        x = F.relu(self.fc1(x))
        # FC-2, then perform ReLU non-linearity
        x = F.relu(self.fc2(x))
        # FC-3
        x = self.fc3(x)
        
        return x
        
        
        ###########################
        #### End of your codes ####
        ###########################


In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    tmp_loss  = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = 'CE'
        if loss == 'CE':
            # CrossEntropy Loss
            loss_fn = nn.CrossEntropyLoss()
        if loss == 'MSE':
            # MSE Loss
            # Prepare for one-hot labels
            y_onehot = target.numpy()
            y_onehot = (np.arange(10) == y_onehot[:,None]).astype(np.float32)
            target = torch.from_numpy(y_onehot)
            loss_fn = nn.MSELoss()

        loss_ = loss_fn(output, target)
        loss_.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss_.item()))
            tmp_loss.append(loss_.item())
    return tmp_loss


In [ ]:
def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
def main():
    time0 = time.time()
    # Training settings
    training_batch_size = 25
    test_batch_size = 20
    epochs = 15
    lr = 0.05
    save_model = True
    use_cuda = True
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=training_batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)
    
    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.292526
Train Epoch: 1 [250/60000 (0%)]	Loss: 2.270250
Train Epoch: 1 [500/60000 (1%)]	Loss: 2.045400
Train Epoch: 1 [750/60000 (1%)]	Loss: 1.510367
Train Epoch: 1 [1000/60000 (2%)]	Loss: 1.521059
Train Epoch: 1 [1250/60000 (2%)]	Loss: 0.315876
Train Epoch: 1 [1500/60000 (2%)]	Loss: 0.610526
Train Epoch: 1 [1750/60000 (3%)]	Loss: 0.873047
Train Epoch: 1 [2000/60000 (3%)]	Loss: 1.052068
Train Epoch: 1 [2250/60000 (4%)]	Loss: 1.179248
Train Epoch: 1 [2500/60000 (4%)]	Loss: 0.483419
Train Epoch: 1 [2750/60000 (5%)]	Loss: 0.746109
Train Epoch: 1 [3000/60000 (5%)]	Loss: 0.549983
Train Epoch: 1 [3250/60000 (5%)]	Loss: 0.403804
Train Epoch: 1 [3500/60000 (6%)]	Loss: 0.311576
Train Epoch: 1 [3750/60000 (6%)]	Loss: 0.337518
Train Epoch: 1 [4000/60000 (7%)]	Loss: 1.158552
Train Epoch: 1 [4250/60000 (7%)]	Loss: 1.029981
Train Epoch: 1 [4500/60000 (8%)]	Loss: 0.626798
Train Epoch: 1 [4750/60000 (8%)]	Loss: 0.645364
Train Epoch: 1 [5000/60000 (8%)]	Loss: 0.80259